In [3]:
!pip install pyspark
!pip install pandas
import pandas as pd
global_train_df = pd.read_csv("../MATHM0029_2024_TB-1/global_train_df.csv")
global_test_df = pd.read_csv("../MATHM0029_2024_TB-1/global_test_df.csv")


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


# Global only 3 Variables

In [4]:
from pyspark.sql import SparkSession
from pyspark.ml.recommendation import ALS
from pyspark.sql.functions import col, collect_list
from pyspark.ml.evaluation import RegressionEvaluator

# Initialize Spark session
spark = SparkSession.builder.appName("RecommenderSystem").getOrCreate()

# Prepare the data
columns = ["User ID", "Item ID", "Rating"]
global_train_spark_df = spark.createDataFrame(global_train_df[columns])
global_test_spark_df = spark.createDataFrame(global_test_df[columns])

# ALS Model Setup
als = ALS(
    userCol="User ID",
    itemCol="Item ID",
    ratingCol="Rating",
    maxIter=10,
    regParam=0.1,
    rank=10,
    coldStartStrategy="drop"  # Prevent NaN predictions
)

# Train the model
als_model = als.fit(global_train_spark_df)

# Generate Recommendations for All Users
user_recommendations = als_model.recommendForAllUsers(10)  # Top 10 recommendations per user

# Evaluate the model with RMSE on test set
predictions = als_model.transform(global_test_spark_df)
evaluator = RegressionEvaluator(
    metricName="rmse",
    labelCol="Rating",
    predictionCol="prediction"
)
rmse = evaluator.evaluate(predictions)
print(f"RMSE: {rmse}")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/18 16:05:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


RMSE: 1.0298879131561742


## Performance Metrics

In [5]:
from pyspark.sql.functions import col, expr, collect_list, explode
from pyspark.sql import Window
import numpy as np

# Extract actual items for each user in the test set
actual_items = global_test_spark_df.groupBy("User ID").agg(
    collect_list("Item ID").alias("actual_items")
)

# Generate top-K recommendations for each user
K = 10
recommendations = als_model.recommendForAllUsers(K)

# Extract recommended items as a list
recommendations = recommendations.select(
    col("User ID"),
    explode(col("recommendations")).alias("rec")  # Exploding the array into individual rows
).select(
    col("User ID"),
    col("rec.Item ID").alias("recommended_items"),  # Accessing the Item ID from the exploded struct
    col("rec.rating").alias("rating")  # If you want to access the rating as well
)


# Join actual items with recommendations
eval_df = recommendations.join(actual_items, on="User ID")

# Define helper functions for metrics
def precision_at_k(recommended, actual, k):
    recommended_k = set(recommended[:k])
    actual_set = set(actual)
    return len(recommended_k & actual_set) / k

def recall_at_k(recommended, actual, k):
    recommended_k = set(recommended[:k])
    actual_set = set(actual)
    return len(recommended_k & actual_set) / len(actual_set) if actual_set else 0

def ndcg_at_k(recommended, actual, k):
    actual_set = set(actual)
    dcg = sum((1 / np.log2(idx + 2)) if item in actual_set else 0 
              for idx, item in enumerate(recommended[:k]))
    idcg = sum((1 / np.log2(idx + 2)) for idx in range(min(len(actual_set), k)))
    return dcg / idcg if idcg > 0 else 0

# Apply metrics for each user
def calculate_metrics(row, k):
    recommended = row["recommended_items"]
    actual = row["actual_items"]
    return (precision_at_k(recommended, actual, k),
            recall_at_k(recommended, actual, k),
            ndcg_at_k(recommended, actual, k))

# Add metrics to the DataFrame
metrics = eval_df.rdd.map(lambda row: (row["User ID"], *calculate_metrics(row, K))).toDF(
    ["User ID", "Precision@K", "Recall@K", "NDCG@K"]
)

# Compute averages across users
avg_metrics = metrics.selectExpr("avg(`Precision@K`) as Precision@K",
                                  "avg(`Recall@K`) as Recall@K",
                                  "avg(`NDCG@K`) as NDCG@K").collect()[0]

print(f"Precision@{K}: {avg_metrics['Precision@K']}")
print(f"Recall@{K}: {avg_metrics['Recall@K']}")
print(f"NDCG@{K}: {avg_metrics['NDCG@K']}")


24/11/18 16:06:07 ERROR Executor: Exception in task 0.0 in stage 138.0 (TID 370)
org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/venv/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 1247, in main
    process()
  File "/opt/venv/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 1239, in process
    serializer.dump_stream(out_iter, outfile)
  File "/opt/venv/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 274, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/opt/venv/lib/python3.10/site-packages/pyspark/rdd.py", line 2849, in takeUpToNumLeft
    yield next(iterator)
  File "/opt/venv/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/util.py", line 83, in wrapper
    return f(*args, **kwargs)
  File "/tmp/ipykernel_195/1988766562.py", line 55, in <lambda>
  File "/tmp/ipykernel_195/198876656

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 138.0 failed 1 times, most recent failure: Lost task 0.0 in stage 138.0 (TID 370) (noteable executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/venv/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 1247, in main
    process()
  File "/opt/venv/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 1239, in process
    serializer.dump_stream(out_iter, outfile)
  File "/opt/venv/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 274, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/opt/venv/lib/python3.10/site-packages/pyspark/rdd.py", line 2849, in takeUpToNumLeft
    yield next(iterator)
  File "/opt/venv/lib/python3.10/site

# Global all Variables

In [15]:
from pyspark.sql.functions import collect_list, mean, array

global_train_spark2_df = spark.createDataFrame(global_train_df)
global_test_spark2_df = spark.createDataFrame(global_test_df)

# List of genre columns
genre_columns = [
    "Action", "Adeventure", "Animation", "Childrens", "Comedy", "Crime", 
    "Documentary", "Drama", "Fantasy", "Film-Noir", "Horror", "Musical", 
    "Mystery", "Romance", "Sci-Fi", "Thriller", "War", "Western"
]

# Step 1: User Features
user_features = global_train_spark2_df.groupBy("User ID").agg(
    mean("Age").alias("avg_age"),
    collect_list("Gender").alias("genders")
)

# Step 2: Item Features
# Aggregate genre features into a vector
item_features = global_train_spark2_df.select(
    "Item ID",
    array(*[col(genre) for genre in genre_columns]).alias("genre_vector")
).distinct()

# Aggregate other item-level features
item_features = item_features.join(
    global_train_spark2_df.groupBy("Item ID").agg(
        mean("Release Date").alias("avg_release_year")
    ),
    on="Item ID",
    how="left"
)

# Step 3: Join Features with Training Data
train_with_features = global_train_spark2_df.join(user_features, on="User ID", how="left") \
                                    .join(item_features, on="Item ID", how="left")

# Display the prepared training data
train_with_features.show(5, truncate=False)


24/11/18 16:29:46 WARN TaskSetManager: Stage 139 contains a task of very large size (5955 KiB). The maximum recommended task size is 1000 KiB.
24/11/18 16:29:47 WARN TaskSetManager: Stage 140 contains a task of very large size (5955 KiB). The maximum recommended task size is 1000 KiB.
24/11/18 16:29:48 WARN TaskSetManager: Stage 141 contains a task of very large size (5955 KiB). The maximum recommended task size is 1000 KiB.
24/11/18 16:29:49 WARN TaskSetManager: Stage 142 contains a task of very large size (5955 KiB). The maximum recommended task size is 1000 KiB.


+-------+-------+------+---------+---+------+----------+--------+-----------------------+------------+--------------------------------------------------------------+-------+------+----------+---------+---------+------+-----+-----------+-----+-------+---------+------+-------+-------+-------+------+--------+---+-------+-------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------+----------------+
|Item ID|User ID|Rating|timestamp|Age|Gender|Occupation|zip code|Movie Title            |Release Date|URL                                                           |Unknown|Action|Adeventure|Animation|Childrens|Comedy|Crime|Documentary|Drama|Fantasy|Film-Noir|Horror|Musical|Mystery|Romance|Sci-Fi|Thriller|War|Western|avg_age|genders          

In [26]:
!pip install lightfm --no-build-isolation
from lightfm import LightFM
from lightfm.data import Dataset

# Prepare data for LightFM
# Extract unique users and items
unique_users = train_with_features.select("User ID").distinct().rdd.flatMap(lambda x: x).collect()
unique_items = train_with_features.select("Item ID").distinct().rdd.flatMap(lambda x: x).collect()

# Step 1: Define Dataset
dataset = Dataset()
dataset.fit(
    users=unique_users,
    items=unique_items,
    user_features=["avg_age", "genders"],  # User features prepared earlier
    item_features=genre_columns + ["avg_release_year"]  # Item features include genres and release year
)

# Step 2: Build Interaction Matrix
interaction_rdd = train_with_features.select("User ID", "Item ID", "Rating").rdd
interactions, weights = dataset.build_interactions([
    (row["User ID"], row["Item ID"], row["Rating"]) for row in interaction_rdd.collect()
])

# Step 3: Build User Features
user_features_rdd = user_features.rdd
user_features_matrix = dataset.build_user_features([
    (row["User ID"], {"avg_age": row["avg_age"], "genders": row["genders"]}) for row in user_features_rdd.collect()
])

# Step 4: Build Item Features
item_features_rdd = item_features.select("Item ID", "genre_vector", "avg_release_year").rdd
item_features_matrix = dataset.build_item_features([
    (row["Item ID"], row["genre_vector"] + [row["avg_release_year"]]) for row in item_features_rdd.collect()
])

# Step 5: Train LightFM Model
model = LightFM(loss='warp')  # Weighted Approximate-Rank Pairwise (WARP) loss for ranking tasks
model.fit(
    interactions,
    user_features=user_features_matrix,
    item_features=item_features_matrix,
    epochs=10
)


  Using cached lightfm-1.17.tar.gz (316 kB)
  Preparing metadata (pyproject.toml) ... error
  error: subprocess-exited-with-error
  
  × Preparing metadata (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [16 lines of output]
      Traceback (most recent call last):
        File "/opt/venv/lib/python3.10/site-packages/pip/_vendor/pyproject_hooks/_in_process/_in_process.py", line 353, in <module>
          main()
        File "/opt/venv/lib/python3.10/site-packages/pip/_vendor/pyproject_hooks/_in_process/_in_process.py", line 335, in main
          json_out['return_val'] = hook(**hook_input['kwargs'])
        File "/opt/venv/lib/python3.10/site-packages/pip/_vendor/pyproject_hooks/_in_process/_in_process.py", line 149, in prepare_metadata_for_build_wheel
          return hook(metadata_directory, config_settings)
        File "/opt/venv/lib/python3.10/site-packages/setuptools/build_meta.py", line 174, in prepare_metadata_for_build_wheel
          self.run_setup()
       

ModuleNotFoundError: No module named 'lightfm'